In [28]:
## scratch pad for citibike data set! exciting! get to break out my pandas knowledge
import pandas as pd
print(pd.__version__) #fortunately, latest version of pandas allows direct import from zip
import zipfile
import requests
import io
import time

0.19.0


In [2]:
## information on filename
citidir = "https://s3.amazonaws.com/tripdata/"
citi_extension = "-citibike-tripdata"
year = 2015
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [4]:
## store all data first in dictionary format using months as keys
d = {}

## download monthly data from the web for each month
for mth in months:
    #create filenames for given month
    filename = str(year) + mth + citi_extension
    citibike_zip_filename = citidir + filename + ".zip"
    citibike_csv_filename = filename + ".csv"
    print(citibike_zip_filename)
    
    #find file and use read_csv to save in dictionary of dataframes
    requested_file = requests.get(citibike_zip_filename)
    z = zipfile.ZipFile(io.BytesIO(requested_file.content))
    d[mth] = pd.read_csv(z.open(citibike_csv_filename), header=0, sep=',', quotechar='"')

https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201502-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201503-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201504-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201505-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201506-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201507-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201508-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201510-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201511-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201512-citibike-tripdata.zip


In [64]:
## Concatenate into single yearly dataframe
df = pd.DataFrame()
tst=0

## append results of each month
for mth in months:
    tst += len(d[mth].index)
    print(tst)
    df = df.append(d[mth], ignore_index=True)

285552
482482
824308
1476698
2438684
3379903
4465579
5644623
6934322
8146599
9133844
9937969


In [75]:
print(df.head())

   tripduration      starttime       stoptime  start station id  \
0          1346  1/1/2015 0:01  1/1/2015 0:24               455   
1           363  1/1/2015 0:02  1/1/2015 0:08               434   
2           346  1/1/2015 0:04  1/1/2015 0:10               491   
3           182  1/1/2015 0:04  1/1/2015 0:07               384   
4           969  1/1/2015 0:05  1/1/2015 0:21               474   

        start station name  start station latitude  start station longitude  \
0          1 Ave & E 44 St               40.750020               -73.969053   
1          9 Ave & W 18 St               40.743174               -74.003664   
2     E 24 St & Park Ave S               40.740964               -73.986022   
3  Fulton St & Waverly Ave               40.683178               -73.965964   
4          5 Ave & E 29 St               40.745168               -73.986831   

   end station id             end station name  end station latitude  \
0             265     Stanton St & Chrystie St    

In [74]:
df["tripduration"].median()

629.0

In [96]:
df['same_start_end'] = pd.Series(df["start station id"] == df["end station id"], index=df.index)

In [103]:
qq = df['same_start_end'].value_counts(normalize = True)

In [106]:
print(qq[True])

0.0223583913373


In [90]:
same_start_end = (df["start station id"] == df["end station id"])

In [94]:
print(sum(same_start_end))
print(sum(same_start_end)/len(same_start_end))

222197
0.0223583913373


In [ ]:
df[df["ischild"] == df].value_counts(normalize = True)

In [62]:
print(d['12'].shape)

(804125, 15)


In [47]:
df = df.append(d['01'], ignore_index=True)
print(df.shape)

(285552, 15)


In [33]:
df.shape

(0, 0)

In [9]:
vertical_stack = pd.concat([d['01'], d['02']], axis=0)

In [14]:
vertical_stack.shape

(482482, 15)

In [15]:
ho_stack = pd.concat([d['01'], d['02']], axis=1)

In [27]:
ho_stack.iloc[[250000]]

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
250000,427,1/25/2015 18:18,1/25/2015 18:25,527,E 33 St & 2 Ave,40.744023,-73.976056,440,E 45 St & 3 Ave,40.752554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
for mth in months:
    citi_file = citidir + str(year) + mth + citi_extension + ".zip"
    print(citi_file)
    df2 = pd.read_csv(citi_file, compression='zip', header=1, sep=',', quotechar='"')

https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip


BadZipFile: File is not a zip file

In [107]:
print(df.head())

   tripduration      starttime       stoptime  start station id  \
0          1346  1/1/2015 0:01  1/1/2015 0:24               455   
1           363  1/1/2015 0:02  1/1/2015 0:08               434   
2           346  1/1/2015 0:04  1/1/2015 0:10               491   
3           182  1/1/2015 0:04  1/1/2015 0:07               384   
4           969  1/1/2015 0:05  1/1/2015 0:21               474   

        start station name  start station latitude  start station longitude  \
0          1 Ave & E 44 St               40.750020               -73.969053   
1          9 Ave & W 18 St               40.743174               -74.003664   
2     E 24 St & Park Ave S               40.740964               -73.986022   
3  Fulton St & Waverly Ave               40.683178               -73.965964   
4          5 Ave & E 29 St               40.745168               -73.986831   

   end station id             end station name  end station latitude  \
0             265     Stanton St & Chrystie St    

In [137]:
bikes = df.groupby(df['bikeid'])
start_stations = df['start station id'].groupby(df['bikeid'])
end_stations = df['end station id'].groupby(df['bikeid'])

In [139]:
qq = bikes[['start station id']]

In [140]:
list(qq)

[(14529, 1524        326
  1793        497
  5715        496
  8123        195
  8360        250
  8642        358
  8812        435
  9156        466
  29006       362
  40268       456
  41290       498
  42450       474
  46917       478
  49233       520
  52693       379
  64762       468
  66571       465
  69884       348
  70780       128
  71709       508
  74789       362
  74959       167
  92747       465
  93800       359
  94762       479
  95286       512
  95408       489
  95589       515
  95984       434
  97689       357
             ... 
  9673414     499
  9673468     499
  9683359     503
  9691099     505
  9702844     510
  9704054     511
  9704064     511
  9710968     513
  9718734     517
  9721447     517
  9727077     519
  9732240     519
  9738998     521
  9740958     521
  9742686     522
  9748180     523
  9767093     531
  9771969     533
  9775500     536
  9778113     537
  9785418     540
  9786726     545
  9856904    3134
  9860888    3137
  9

In [141]:
list(start_stations)

[(14529, 1524        326
  1793        497
  5715        496
  8123        195
  8360        250
  8642        358
  8812        435
  9156        466
  29006       362
  40268       456
  41290       498
  42450       474
  46917       478
  49233       520
  52693       379
  64762       468
  66571       465
  69884       348
  70780       128
  71709       508
  74789       362
  74959       167
  92747       465
  93800       359
  94762       479
  95286       512
  95408       489
  95589       515
  95984       434
  97689       357
             ... 
  9673414     499
  9673468     499
  9683359     503
  9691099     505
  9702844     510
  9704054     511
  9704064     511
  9710968     513
  9718734     517
  9721447     517
  9727077     519
  9732240     519
  9738998     521
  9740958     521
  9742686     522
  9748180     523
  9767093     531
  9771969     533
  9775500     536
  9778113     537
  9785418     540
  9786726     545
  9856904    3134
  9860888    3137
  9

In [135]:
df.loc[1793]

tripduration                              829
starttime                      1/1/2015 13:11
stoptime                       1/1/2015 13:24
start station id                          497
start station name         E 17 St & Broadway
start station latitude                 40.737
start station longitude              -73.9901
end station id                            515
end station name             W 43 St & 10 Ave
end station latitude                  40.7601
end station longitude                -73.9946
bikeid                                  14529
usertype                           Subscriber
birth year                               1991
gender                                      1
same_start_end                          False
Name: 1793, dtype: object

In [143]:
rr = set(list(start_stations))

TypeError: 'Series' objects are mutable, thus they cannot be hashed